In [1]:
import torch

In [2]:
torch.__version__

'2.3.1+cu121'

In [3]:
import transformers

In [4]:
transformers.__version__

'4.42.4'

In [5]:
from transformers import AutoTokenizer

In [6]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [14]:
sentence = "I skip across the"

In [15]:
input_ids = tokenizer(sentence, return_tensors='pt').input_ids

In [16]:
input_ids

tensor([[   40, 14267,  1973,   262]])

In [17]:
# words --> tokens --> Unique ID --> vector embed

In [18]:
for token_id in input_ids[0]:
  print(tokenizer.decode(token_id))

I
 skip
 across
 the


In [19]:
from transformers import AutoModelForCausalLM

In [20]:
gpt2 = AutoModelForCausalLM.from_pretrained("gpt2")

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [21]:
outputs = gpt2(input_ids)

In [22]:
outputs.logits.shape

torch.Size([1, 4, 50257])

In [23]:
final_logits = gpt2(input_ids).logits[0, -1]

In [24]:
final_logits

tensor([-87.5486, -86.0567, -88.3320,  ..., -88.2509, -90.5480, -86.1209],
       grad_fn=<SelectBackward0>)

In [25]:
final_logits.argmax() #Token ID --> Index Location Logits

tensor(1627)

In [26]:
tokenizer.decode(final_logits.argmax())

' line'

In [27]:
top10_logits = torch.topk(final_logits, 10)

In [28]:
for index in top10_logits.indices:
  print(tokenizer.decode(index))

 line
 street
 river
 room
 pond
 bridge
 border
 country
 road
 board


In [29]:
top10 = torch.topk(final_logits.softmax(dim=0), 10)

In [31]:
for value, index in zip(top10.values, top10.indices):
  print(f"{tokenizer.decode(index)} -- {value.item(): .1%}") #Chance for another word being after 'the' in sentence

 line --  3.6%
 street --  2.7%
 river --  2.2%
 room --  1.9%
 pond --  1.8%
 bridge --  1.7%
 border --  1.7%
 country --  1.7%
 road --  1.6%
 board --  0.9%
